# Modeling the success of a Portuguese bank's marketing campaign

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_validate, GridSearchCV, train_test_split, ParameterSampler
from xgboost import XGBClassifier, DMatrix, cv
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import roc_auc_score, plot_precision_recall_curve, plot_roc_curve
from time import time
from scipy.stats import uniform
from functools import partial
from dask.distributed import Client
import mlflow
import os

Data is taken from: https://www.openml.org/d/1461 and is related to direct marketing campaigns of a Portuguese banking institution.
See the link for details.

The goal is to predict whether a client will subscribe a term deposit or not (variable 'target' below).

In [2]:
data = pd.read_csv('data.csv', 
                   header=0,
                   names=['age', 'job', 'marital_status', 'education', 
                          'default', 'balance', 'housing', 'loan', 
                          'contact', 'day', 'month', 'duration',
                          'campaign', 'previous_days', 'previous_contacts', 'previous_outcome',
                          'target'],
                   dtype={'age': 'int16',
                          'job': 'category',
                          'marital_status': 'category',
                          'education': 'category',
                          'default': 'category',
                          'balance': 'int32',
                          'housing': 'category',
                          'loan': 'category',
                          'contact': 'category',
                          'day': 'int16',
                          'month': 'category',
                          'duration': 'int16',
                          'campaign': 'int16',
                          'previous_days': 'int16',
                          'previous_contacts': 'int16',
                          'previous_outcome': 'category'
                         }
                  )

data['target'] = np.where(data['target']==1, 0, 1)

data.head()

,age,job,marital_status,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous_days,previous_contacts,previous_outcome,target
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


## Preprocessing categorical columns

In order to run typical machine learning algorithms, we need to transform the categorical columns' format form text to numbers.

Here are the criteria we use to choose how to preprocess the categorical columns:
- if there is no obvious intrinsic order and more than two possible values, we use a **One Hot Encoder**
- if there is an obvious intrinsic order and more than two possible values, we use an **Ordinal Encoder** and enforce the order of the values
- if there are just two possible values, we use an **Ordinal Encoder** and we don't care about the order

In [3]:
encoders = make_column_transformer((OneHotEncoder(), ['job', 'marital_status', 'contact']),
                                  (OrdinalEncoder(), ['default', 'housing', 'loan']),
                                  (OrdinalEncoder([['unknown', 'primary', 'secondary', 'tertiary'], 
                                                   ['failure', 'unknown', 'other', 'success'],
                                                   ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']]), 
                                                  ['education', 'previous_outcome', 'month']),
                                  remainder='passthrough')

data_encoded = pd.DataFrame(encoders.fit_transform(data))

In [4]:
data_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,58.0,2143.0,5.0,261.0,1.0,-1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,4.0,44.0,29.0,5.0,151.0,1.0,-1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,33.0,2.0,5.0,76.0,1.0,-1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,47.0,1506.0,5.0,92.0,1.0,-1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,33.0,1.0,5.0,198.0,1.0,-1.0,0.0,0.0


Although very complete already, the _sklearn_ module sometimes requires some manual adjustments.

In this case, the output of `encoders.fit_transform` is a numpy array, and we completely lose the information on the feature names.

So we write a small function to correct that:

In [5]:
import re

def _extract_feature_names_(data, transformer_tuple):
    
    if hasattr(transformer_tuple[1], 'get_feature_names'):
        res = transformer_tuple[1].get_feature_names().tolist()
        res_str = ' '.join(res)
        for i, f in enumerate(transformer_tuple[2]):
            res_str = re.sub(f'x{i}_', f + '_', res_str)
        return res_str.split(' ')
    
    elif transformer_tuple[0]=='remainder' and transformer_tuple[1]=='passthrough':
        return data.columns[transformer_tuple[2]].values.tolist()
    
    else:
        return transformer_tuple[2]
    
def extract_feature_names(data, encoders):
    return [item for transformer_tuple in encoders.transformers_ for item in _extract_feature_names_(data, transformer_tuple)]

In [6]:
data_encoded.columns = extract_feature_names(data, encoders)
data_encoded.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,previous_outcome,month,age,balance,day,duration,campaign,previous_days,previous_contacts,target
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,58.0,2143.0,5.0,261.0,1.0,-1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,4.0,44.0,29.0,5.0,151.0,1.0,-1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,33.0,2.0,5.0,76.0,1.0,-1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,47.0,1506.0,5.0,92.0,1.0,-1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,33.0,1.0,5.0,198.0,1.0,-1.0,0.0,0.0


Let's just validate that the values are as expected:

In [52]:
data_encoded.iloc[0]

job_admin.                    0.0
job_blue-collar               0.0
job_entrepreneur              0.0
job_housemaid                 0.0
job_management                1.0
job_retired                   0.0
job_self-employed             0.0
job_services                  0.0
job_student                   0.0
job_technician                0.0
job_unemployed                0.0
job_unknown                   0.0
marital_status_divorced       0.0
marital_status_married        1.0
marital_status_single         0.0
contact_cellular              0.0
contact_telephone             0.0
contact_unknown               1.0
default                       0.0
housing                       1.0
loan                          0.0
education                     3.0
previous_outcome              1.0
month                         4.0
age                          58.0
balance                    2143.0
day                           5.0
duration                    261.0
campaign                      1.0
previous_days 

In [53]:
data.iloc[0]

age                          58
job                  management
marital_status          married
education              tertiary
default                      no
balance                    2143
housing                     yes
loan                         no
contact                 unknown
day                           5
month                       may
duration                    261
campaign                      1
previous_days                -1
previous_contacts             0
previous_outcome        unknown
target                        0
Name: 0, dtype: object

It looks good. We can now create our training and testing splits.

## Train/test split

In [7]:
X = data_encoded[[c for c in data_encoded.columns if c != 'target']]
y = data_encoded['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

### Approximate number of trees needed for a given learning rate

In [55]:
dtrain = DMatrix(X_train, label=y_train)
cv(params={'learning_rate': 0.1, 'max_depth': 6}, dtrain=dtrain, num_boost_round=250, nfold=5, early_stopping_rounds=20)

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,0.464483,0.000199,0.464989,0.000202
1,0.433480,0.000374,0.434522,0.000479
2,0.406577,0.000474,0.408132,0.000775
3,0.383305,0.000622,0.385434,0.000959
4,0.363167,0.000653,0.365876,0.001269
...,...,...,...,...
107,0.215924,0.001593,0.251621,0.006220
108,0.215783,0.001647,0.251636,0.006204
109,0.215551,0.001588,0.251596,0.006298
110,0.215397,0.001652,0.251577,0.006256


In [8]:
folds = KFold(5, random_state=5, shuffle=True)

In [58]:
xgb_model = XGBClassifier(n_estimators=112, learning_rate=0.1)

param_grid = {
    'max_depth': [4, 5, 6, 7, 5],
    'subsample': [0.5, 0.75],
    'colsample_bytree': [0.5, 0.75]
}

grid_search = GridSearchCV(xgb_model, param_grid, scoring='roc_auc', cv=folds, verbose=1, n_jobs=-1)

start_time = time()
grid_search.fit(X_train, y_train)
end_time = time()

print('Elapsed:', end_time - start_time, 'sec.')

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.7min finished


Elapsed: 348.5636031627655 sec.


In [ ]:
grid_search.cv_results_

In [60]:
best_param_idx = int(np.argwhere(grid_search.cv_results_['rank_test_score']==1))
best_params = grid_search.cv_results_['params'][best_param_idx]
best_score = grid_search.cv_results_['mean_test_score'][best_param_idx]

print('Best score:', best_score)
print()
print('Best params:', best_params)

Best score: 0.9341800908705963

Best params: {'colsample_bytree': 0.5, 'max_depth': 7, 'subsample': 0.75}


### Test AUC

In [ ]:
y_preds = grid_search.predict_proba(X_test)[:, 1]

In [ ]:
roc_auc_score(y_test, y_preds)

In [ ]:
plot_roc_curve(grid_search, X_test, y_test)

In [ ]:
plot_precision_recall_curve(grid_search, X_test, y_test)

# Random search

In [9]:
sampler = ParameterSampler({'gamma': uniform(0, 5),
                            'max_depth': list(range(4, 10)),
                            'min_child_weight': uniform(0, 10),
                            'subsample': uniform(0.5, 0.5),
                            'colsample_bytree': uniform(0.5, 0.5),
                            'reg_lambda': [0, 0.25, 0.5, 0.75, 1],
                            'reg_alpha': [0, 0.25, 0.5, 0.75, 1]},
                          n_iter = 100,
                          random_state=123)

In [10]:
param_list = list(sampler)

print(len(param_list))

param_list[0:2]

100


[{'colsample_bytree': 0.8482345927989308,
  'gamma': 1.4306966747518972,
  'max_depth': 8,
  'min_child_weight': 6.908848550268617,
  'reg_alpha': 0.25,
  'reg_lambda': 0.75,
  'subsample': 0.7455594667162986},
 {'colsample_bytree': 0.8900138809560396,
  'gamma': 2.0546218635450946,
  'max_depth': 5,
  'min_child_weight': 4.809319014843609,
  'reg_alpha': 0.25,
  'reg_lambda': 0,
  'subsample': 0.6715890080754348}]

In [11]:
def fit_model(params, X, y, cv):
    model = XGBClassifier(n_estimators=112, learning_rate=0.1, **params)
    return cross_validate(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)

## Local mode

In [37]:
start_time = time()

res_local =[fit_model(param, X=X_train, y=y_train, cv=folds) for param in param_list]

end_time = time()

In [39]:
end_time - start_time

3004.3847873210907

In [61]:
res_local[0:2]

[{'fit_time': array([38.19000626, 37.93771839, 38.75668907, 38.13801646, 38.32303309]),
  'score_time': array([0.21663976, 0.23832846, 0.24399114, 0.23998833, 0.23300052]),
  'test_score': array([0.93533667, 0.93073737, 0.93892571, 0.93589523, 0.93177839])},
 {'fit_time': array([28.62689805, 28.67320061, 27.97429037, 28.5950017 , 27.97248459]),
  'score_time': array([0.1549468 , 0.12305331, 0.15894413, 0.16352749, 0.15199971]),
  'test_score': array([0.93259728, 0.9290668 , 0.93632107, 0.93382456, 0.92991526])}]

In [64]:
score_res = [r['test_score'].mean() for r in res_local]

best_score = max(score_res)

best_score_idx = np.argmax(best_score)

best_params = param_list[best_score_idx]

print('Best score found:', best_score)
print('Best params found:', best_params)

Best score found: 0.9361782651923466
Best params found: {'colsample_bytree': 0.8482345927989308, 'gamma': 1.4306966747518972, 'max_depth': 8, 'min_child_weight': 6.908848550268617, 'reg_alpha': 0.25, 'reg_lambda': 0.75, 'subsample': 0.7455594667162986}


## Distributed mode

In [ ]:
client = Client('ec2-3-84-88-28.compute-1.amazonaws.com:40393')

In [27]:
start_time = time()

futures = client.map(fit_model, param_list, X=X_train, y=y_train, cv=folds)
res = client.gather(futures)

end_time = time()

In [28]:
(end_time - start_time)

681.1898059844971

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


# Adding MLflow to the mix

After setting up MLflow's tracking server on EC2, make sure you can connect to it:

In [15]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'mlflow'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'mlflow'

# you can set your tracking server URI programmatically:
mlflow.set_tracking_uri('http://ec2-52-207-217-8.compute-1.amazonaws.com')

mlflow.set_experiment("BankMarketing")

INFO: 'BankMarketing' does not exist. Creating a new experiment


Our updated `fit_model` function:

In [20]:
def fit_model_mlflow(params, X, y, cv):
    with mlflow.start_run():
        for param, value in params.items():
            mlflow.log_param(param, value)
            
        model = XGBClassifier(n_estimators=112, learning_rate=0.1, **params)
        
        result = cross_validate(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
        
        avg_auc = result['test_score'].mean()
        mlflow.log_metric('auc', avg_auc)
        
        return result

We need to tell the worker nodes about MLflow:

In [23]:
def init_mlflow():
    os.environ['MLFLOW_TRACKING_USERNAME'] = 'mlflow'
    os.environ['MLFLOW_TRACKING_PASSWORD'] = 'mlflow'

    # you can set your tracking server URI programmatically:
    mlflow.set_tracking_uri('http://ec2-52-207-217-8.compute-1.amazonaws.com')

    mlflow.set_experiment("BankMarketing")

In [24]:
client = Client('ec2-52-90-247-177.compute-1.amazonaws.com:32957')
client.run(init_mlflow)

{'tcp://172.31.17.164:42747': None,
 'tcp://172.31.19.201:39583': None,
 'tcp://172.31.19.235:39771': None,
 'tcp://172.31.19.235:41867': None}

## It's go time !

In [25]:
start_time = time()

futures = client.map(fit_model_mlflow, param_list, X=X_train, y=y_train, cv=folds)
res = client.gather(futures)

end_time = time()